# Check Payroll
This is the first notebook in a group of notebooks to check results of my code for the Oct 29th methodology check.
This notebook is to check payroll for each agency. The reason I need a new notebook is twofold: first to get payroll without agency correction, and second to get hidden payroll for chelsea, boston pd

In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
import pandas as pd
import sys
import os
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={"figure.figsize": (20, 18), "lines.linewidth": 5})
sns.set_style("darkgrid")

In [4]:
sys.path.insert(0, "/Users/alexanderweinstein/Documents/Harris/Summer2020/Carceral_Budgeting/Exploratory")
from Initialize_Agencies_2Ver import get_agencies

In [5]:
from Agency_Classes import Agency, StateAgency, PoliceDepartment

In [6]:
yr = list(range(2016,2020))

In [7]:
agencies = get_agencies(yr)

# Stated Payroll
For state agencies, any spending with object code starting with "A" from expenditures dataset. 

In [8]:
stated_payroll_df = pd.DataFrame(columns=yr, index=agencies.keys(), data=0)


In [9]:
for name, agency in agencies.items():
    if isinstance(agency, StateAgency) and name != "MBTA":
        if agency.payroll_expenditures_by_year is None:
            agency.get_final_costs()
        stated_payroll_df.loc[name] = agency.payroll_expenditures_by_year
    if isinstance(agency, PoliceDepartment) and name != "Revere PD":
        stated_payroll_df.loc[name] = agency.budget_summary.loc["Payroll Expenditures", yr]
    if name == "Chelsea PD":
        stated_payroll_df.loc[name, 2016] =agency.budget_summary.loc["Payroll Budget", 2016]
    if name == "Revere PD":
        stated_payroll_df.loc[name] = agency.budget_summary.loc["Payroll Adopted", yr]

In [10]:
# stated_payroll_df.to_csv("Stated_Payroll_Oct23.csv")

# Hidden Payroll
For MBTA, all payroll.
For Cheslea PD, Boston PD payroll from separate employee earnings datasets

In [13]:
hidden_payroll_df = pd.DataFrame(columns=yr, index=agencies.keys(), data=0)

In [28]:
for name in ["Boston PD", "Chelsea PD", "MBTA"]:
    if agencies[name].payroll_expenditures_by_year is None:
        agencies[name].get_final_costs()
    hidden_payroll_df.loc[name] = agencies[name].payroll_expenditures_by_year
hidden_payroll_df.loc["Chelsea PD", 2016] =0

In [36]:
hidden_payroll_df

,2016,2017,2018,2019
trial_court,0.000,0.000,0.000,0.000
CPCS,0.000,0.000,0.000,0.000
DOC,0.000,0.000,0.000,0.000
Suffolk DA,0.000,0.000,0.000,0.000
Suffolk Sheriff,0.000,0.000,0.000,0.000
State_Police,0.000,0.000,0.000,0.000
MBTA,28140924.850,22662116.390,24217763.280,31640635.490
DAA,0.000,0.000,0.000,0.000
MA Sheriff's Association,0.000,0.000,0.000,0.000
Parole Board,0.000,0.000,0.000,0.000


In [37]:
hidden_payroll_df.to_csv("Hidden_Payroll_Oct23.csv")

# Why use pdfs from MBTA site when there is payroll data for each year?
I was using pdfs from https://www.mbta.com/financials/mbta-budget bt I'm not sure I have to do that. See if cthru payroll dataset has payroll for each yeat for MBTA

In [32]:
MBTA = agencies["MBTA"]
pd.set_option("display.max.columns",22)
MBTA.payroll.head(5)

,Unnamed: 0,year,name_last,name_first,department_division,position_title,position_type,service_end_date,pay_total_actual,pay_base_actual,pay_buyout_actual,pay_overtime_actual,pay_other_actual,annual_rate,pay_year_to_date,department_location_zip_code,contract,bargaining_group_no,bargaining_group_title,trans_no,chris,police_pay
0,0,2019,Calascibetta,Jack,Massachusetts Bay Transportation Authority (MBT),"Laborer,Track",Full Time Employee,2019-12-21T00:00:00.000,4883.750,4419.680,0.000,464.070,0.000,45964.620,4883.750,2129.000,MBTA L589 Carmen's Misc (M12),M12,MBTA L589 Carmen's Misc,19592365,MBT,0.000
1,1,2019,Fitzgerald,Sean,Massachusetts Bay Transportation Authority (MBT),Instructor,Full Time Employee,2019-12-21T00:00:00.000,105504.020,93646.800,0.000,11857.220,0.000,93288.000,105504.020,2127.000,MBTA ALL Alliance (M15),M15,MBTA ALL Alliance,2201160,MBT,0.000
2,2,2018,Bell,Whitney,Massachusetts Bay Transportation Authority (MBT),"Officer,Police",Full Time Employee,2018-12-22T00:00:00.000,124245.270,68516.010,0.000,48294.730,7434.530,64591.800,124245.270,2118.000,MBTA Police Officers (M98),M98,MBTA Police Officers,2238626,MBT,124245.270
3,3,2018,Talley,Tara,Massachusetts Bay Transportation Authority (MBT),"Motorperson,Spare RTL Yard",Full Time Employee,2018-12-22T00:00:00.000,19135.800,15491.520,0.000,3644.280,0.000,75707.580,19135.800,2155.000,MBTA L589 Carmen's Non-Misc (M11),M11,MBTA L589 Carmen's Non-Misc,2168543,MBT,0.000
4,4,2019,Burt,Tash,Massachusetts Bay Transportation Authority (MBT),"Officer,Police",Full Time Employee,2019-12-21T00:00:00.000,138943.040,78845.950,0.000,58461.930,1635.160,68723.200,138943.040,2118.000,MBTA Police Officers (M98),M98,MBTA Police Officers,19252181,MBT,138943.040


In [34]:
MBTA.payroll.groupby("year").sum()["police_pay"]

year
2017   15234666.310
2018   24217763.280
2019   31640635.490
2020   11782749.440
Name: police_pay, dtype: float64

In [35]:
MBTA.payroll_expenditures_by_year

year
2016   28140924.850
2017   22662116.390
2018   24217763.280
2019   31640635.490
Name: police_pay, dtype: float64

# Double check

downloaded all 2017 expenditures data for CPCS

In [38]:
check_cpcs = pd.read_csv("cpcs_2019_expenditures.csv")

In [ ]:
pd.set_option("display.max.columns", 25)
check_cpcs["Amount"] = check_cpcs["Amount"].astype(float)

In [55]:
old_payroll_calculation = check_cpcs[check_cpcs["Object_Code"].str[1] == "A"]["Amount"].sum()

In [56]:
new_payroll_calculation = check_cpcs[check_cpcs["Vendor"]=="CPC - SUMMARY PAYROLL"]["Amount"].sum()

In [57]:
new_payroll_calculation - old_payroll_calculation

1126680.1399999931